<a href="https://colab.research.google.com/github/min02yam/R-study/blob/main/%EC%97%B0%EC%8A%B5/stats%2C%20caret%2C%20fastDummies%2C%20DataExplorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**caret** <br>
3.Pre-Processing을 공부하면서<br>
더미코딩(less than full rank parameterized)과 
원핫인코딩(a full set of dummy variables) 정리<br>
2022.06.13

1. `stats`::model.matrix (더미코딩)
     - intercept 생성
2. `caret`::dummyVars (디폴트:원핫인코딩/ 더미코딩)
    - predict으로 생성해야 함, fullRank = FALSE
3. `fastDummies`:: dummy_cols (디폴트:원핫인코딩/ 더미코딩)
    - dummy_rows 가능 , remove_first_dummy=  FALSE
    - select_columns 특정 열만
    - remove_selected_columns=FALSE 대상 변수 살림
4. `DataExplorer`::dummify ( 원핫인코딩)
    - maxcat 맥스 카테고리 개수 
    - select 변수선택
5. 결론
- 원핫인코딩 -> DataExplorer가 제일 빠름
- 여러 옵션 주고싶을 때 -> fastDummies
- 

In [15]:
#install.packages("caret")
#install.packages(c("tidyverse","pacman"))
#install.packages(c("dplyr","DataExplorer","skimr"))
#pacman::p_load("tidyverse", "DataExplorer","skimr")
library(caret)

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




# 1. stats:: dummyVars

In [4]:
#데이터 로드
#install.packages("earth")
df=earth::etitanic

In [9]:
#간단 구조 확인
skim(df)

── Data Summary ────────────────────────
                           Values
Name                       df    
Number of rows             1046  
Number of columns          6     
_______________________          
Column type frequency:           
  factor                   2     
  numeric                  4     
________________________         
Group variables            None  

── Variable type: factor ───────────────────────────────────────────────────────
  skim_variable n_missing complete_rate ordered n_unique
1 pclass                0             1 FALSE          3
2 sex                   0             1 FALSE          2
  top_counts                  
1 3rd: 501, 1st: 284, 2nd: 261
2 mal: 658, fem: 388          

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean     sd    p0 p25 p50 p75 p100
1 survived              0             1  0.408  0.492 0       0   0   1    1
2 age                   0            

Warning message in is.null(text_repr) || nchar(text_repr) == 0L:
“'length(x) = 2 > 1' in coercion to 'logical(1)'”


In [12]:
#베이직
stats::model.matrix(survived ~ ., df) %>% head

,(Intercept),pclass2nd,pclass3rd,sexmale,age,sibsp,parch
1,1,0,0,0,29.0000,0,0
2,1,0,0,1,0.9167,1,2
3,1,0,0,0,2.0000,1,2
4,1,0,0,1,30.0000,1,2
5,1,0,0,0,25.0000,1,2
6,1,0,0,1,48.0000,0,0


# 2. caret::dummyVars사용

In [19]:
#caret::dummyVars사용
dmy= caret::dummyVars(survived~., df)
predict(dmy, newdata=df) %>% head

,pclass.1st,pclass.2nd,pclass.3rd,sex.female,sex.male,age,sibsp,parch
1,1,0,0,1,0,29.0000,0,0
2,1,0,0,0,1,0.9167,1,2
3,1,0,0,1,0,2.0000,1,2
4,1,0,0,0,1,30.0000,1,2
5,1,0,0,1,0,25.0000,1,2
6,1,0,0,0,1,48.0000,0,0


In [22]:
dmy= caret::dummyVars(survived~., data=df, fullRank = TRUE  )
predict(dmy, newdata=df) %>% head

,pclass.2nd,pclass.3rd,sex.male,age,sibsp,parch
1,0,0,0,29.0000,0,0
2,0,0,1,0.9167,1,2
3,0,0,0,2.0000,1,2
4,0,0,1,30.0000,1,2
5,0,0,0,25.0000,1,2
6,0,0,1,48.0000,0,0


# 3. fastDummies

In [23]:
install.packages("fastDummies")
library(fastDummies)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [25]:
#행에 만들수도 있음 
dummy_rows(df) %>% head

,pclass,survived,sex,age,sibsp,parch
,<fct>,<int>,<fct>,<dbl>,<int>,<int>
1,1st,1,female,29.0000,0,0
2,1st,1,male,0.9167,1,2
3,1st,0,female,2.0000,1,2
4,1st,0,male,30.0000,1,2
5,1st,0,female,25.0000,1,2
6,1st,1,male,48.0000,0,0


In [35]:
dummy_rows(.data=df,select_columns = c("pclass", "sex") )   %>% tail  

,pclass,survived,sex,age,sibsp,parch
,<fct>,<int>,<fct>,<dbl>,<int>,<int>
1041,3rd,1,female,15.0,1,0
1042,3rd,0,male,45.5,0,0
1043,3rd,0,female,14.5,1,0
1044,3rd,0,male,26.5,0,0
1045,3rd,0,male,27.0,0,0
1046,3rd,0,male,29.0,0,0


In [46]:
dummy_cols(.data= df, remove_first_dummy=  FALSE, remove_selected_columns=TRUE )    %>% head #원핫인코딩
#지정 안하면 팩터 모두 변형
#remove_first_dummy=  FALSE
#remove_selected_columns=TRUE

,survived,age,sibsp,parch,pclass_1st,pclass_2nd,pclass_3rd,sex_female,sex_male
,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,29.0000,0,0,1,0,0,1,0
2,1,0.9167,1,2,1,0,0,0,1
3,0,2.0000,1,2,1,0,0,1,0
4,0,30.0000,1,2,1,0,0,0,1
5,0,25.0000,1,2,1,0,0,1,0
6,1,48.0000,0,0,1,0,0,0,1


In [40]:
dummy_cols(.data= df, remove_first_dummy=  FALSE, remove_selected_columns=FALSE )  %>% head

,pclass,survived,sex,age,sibsp,parch,pclass_1st,pclass_2nd,pclass_3rd,sex_female,sex_male
,<fct>,<int>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1st,1,female,29.0000,0,0,1,0,0,1,0
2,1st,1,male,0.9167,1,2,1,0,0,0,1
3,1st,0,female,2.0000,1,2,1,0,0,1,0
4,1st,0,male,30.0000,1,2,1,0,0,0,1
5,1st,0,female,25.0000,1,2,1,0,0,1,0
6,1st,1,male,48.0000,0,0,1,0,0,0,1


In [42]:
dummy_cols(.data= df, remove_first_dummy=  TRUE, remove_selected_columns=TRUE )   %>% head  # 더미코딩

,survived,age,sibsp,parch,pclass_2nd,pclass_3rd,sex_male
,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>
1,1,29.0000,0,0,0,0,0
2,1,0.9167,1,2,0,0,1
3,0,2.0000,1,2,0,0,0
4,0,30.0000,1,2,0,0,1
5,0,25.0000,1,2,0,0,0
6,1,48.0000,0,0,0,0,1


[예제] https://github.com/min02yam/R-study/blob/main/%EC%97%B0%EC%8A%B5/fastDummies.ipynb

In [26]:
crime <- data.frame(city = c("SF", "SF", "NYC"),
year = c(1990, 2000, 1990),
crime = 1:3); crime

city,year,crime
<chr>,<dbl>,<int>
SF,1990,1
SF,2000,2
NYC,1990,3


In [27]:
dummy_rows(crime)

city,year,crime
<chr>,<dbl>,<int>
SF,1990,1
SF,2000,2
NYC,1990,3


In [28]:
# Include year column
dummy_rows(crime, select_columns = c("city", "year"))

city,year,crime
<chr>,<dbl>,<int>
SF,1990,1
SF,2000,2
NYC,1990,3
NYC,2000,NA


#4. DataExplorer:: dummify

In [49]:
dummify(df) %>% head

,survived,age,sibsp,parch,pclass_1st,pclass_2nd,pclass_3rd,sex_female,sex_male
,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,29.0000,0,0,1,0,0,1,0
2,1,0.9167,1,2,1,0,0,0,1
3,0,2.0000,1,2,1,0,0,1,0
4,0,30.0000,1,2,1,0,0,0,1
5,0,25.0000,1,2,1,0,0,1,0
6,1,48.0000,0,0,1,0,0,0,1
